# Titanic Passenger Clusters with Mean Shift

In this dataset, we will use the famous Titanic Passengers dataset to test out a mean shift model, and see if the model's clusters can give us any insight into what determined if a passenger survuved or died.

Let's start by importing the linraries we will be using and loading the dataset.

In [1]:
import numpy as np
from sklearn.cluster import MeanShift, KMeans
from sklearn import preprocessing, cross_validation
import pandas as pd
import matplotlib.pyplot as plt


'''
Pclass Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
survival Survival (0 = No; 1 = Yes)
name Name
sex Sex
age Age
sibsp Number of Siblings/Spouses Aboard
parch Number of Parents/Children Aboard
ticket Ticket Number
fare Passenger Fare (British pound)
cabin Cabin
embarked Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
boat Lifeboat
body Body Identification Number
home.dest Home/Destination
'''

df = pd.read_excel('titanic.xls')

C:\Users\Drew\Anaconda3\lib\site-packages\sklearn\cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


Let's make a copy of the df so that we can see values as text instead of ints at the end. We must convert qualifiable values to ints for the model, and wil use the copy of the df to return the text values after. We will make a function for this conversion below.

In [2]:
original_df = pd.DataFrame.copy(df)
df.drop(['body','name'], 1, inplace=True)
df.fillna(0,inplace=True)

def handle_non_numerical_data(df):
    
    columns = df.columns.values

    for column in columns:
        text_digit_vals = {}
        def convert_to_int(val):
            return text_digit_vals[val]

        if df[column].dtype != np.int64 and df[column].dtype != np.float64:
            
            column_contents = df[column].values.tolist()
            unique_elements = set(column_contents) 
            x = 0
            for unique in unique_elements:
                if unique not in text_digit_vals:
                    text_digit_vals[unique] = x
                    x+=1 
            df[column] = list(map(convert_to_int,df[column]))

    return df

We will now run the conversion function on the df, and drop un-needed fields.

In [3]:
df = handle_non_numerical_data(df)
df.drop(['ticket','home.dest'], 1, inplace=True)

Now we will create and train our model.

In [4]:
X = np.array(df.drop(['survived'], 1).astype(float))
X = preprocessing.scale(X)
y = np.array(df['survived'])

clf = MeanShift()
clf.fit(X)

MeanShift(bandwidth=None, bin_seeding=False, cluster_all=True, min_bin_freq=1,
     n_jobs=1, seeds=None)

We will now create a placeholder column for where we will be assigning a cluster group to each passenger. We will then iterate through each passenger and fill in the placeholder column by assigning them the appropriate cluster group. We will then count how many clusters the model created.

In [5]:
labels = clf.labels_
cluster_centers = clf.cluster_centers_

original_df['cluster_group'] = np.nan

for i in range(len(X)):
    original_df['cluster_group'].iloc[i] = labels[i]

n_clusters_ = len(np.unique(labels))
print(n_clusters_)

C:\Users\Drew\Anaconda3\lib\site-packages\pandas\core\indexing.py:194: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  self._setitem_with_indexer(indexer, value)


4


We would like to check if the model's clusters reflect chances of survival in any way. Let's calculate survival rates per cluster.

In [6]:
survival_rates = {}
for i in range(n_clusters_):
    temp_df = original_df[ (original_df['cluster_group']==float(i)) ]
    survival_cluster = temp_df[  (temp_df['survived'] == 1) ]

    survival_rate = len(survival_cluster) / len(temp_df)
    print(i,survival_rate)
    survival_rates[i] = survival_rate
    
#print(survival_rates)

0 0.3723150357995227
1 1.0
2 0.6944444444444444
3 0.1


We should now explore these clusters to see if we can get any valuable insight. We can do this by using a where clause, only pulling data from rows where the cluster group assigned matches what we want.

In [7]:
print(original_df[(original_df['cluster_group']==2)])
print(original_df[(original_df['cluster_group']==2)].describe())

     pclass  survived                                               name  \
10        1         0                             Astor, Col. John Jacob   
11        1         1  Astor, Mrs. John Jacob (Madeleine Talmadge Force)   
16        1         0                           Baxter, Mr. Quigg Edmond   
17        1         1    Baxter, Mrs. James (Helene DeLaudeniere Chaput)   
32        1         1                            Bonnell, Miss. Caroline   
54        1         1                Carter, Master. William Thornton II   
55        1         1                          Carter, Miss. Lucile Polk   
56        1         1                         Carter, Mr. William Ernest   
57        1         1          Carter, Mrs. William Ernest (Lucile Polk)   
61        1         1  Cavendish, Mrs. Tyrell William (Julia Florence...   
97        1         1  Douglas, Mrs. Frederick Charles (Mary Helene B...   
103       1         1                      Endres, Miss. Caroline Louise   
111       1 

We can create separate dataframes using the same logic to separate clusters. From there, we can get even more specific, for example, 1st class passangers in cluster 2. With this info, we can see that 1st class passengers in cluster 2 had a 70% survival rate. Breaking this down further and finding what caused this clustering could then give us some valuable insight.

In [8]:
cluster_2 = original_df[(original_df['cluster_group']==2)]
cluster_2_fc = cluster_2[(cluster_2['pclass']==1)]
cluster_2_fc.describe()
#This shows that the first class passengers in cluster 2 had a 70% survival rate

,pclass,survived,age,sibsp,parch,fare,body,cluster_group
count,36.0,36.000000,36.000000,36.000000,36.000000,36.000000,4.000000,36.0
mean,1.0,0.694444,37.041667,0.944444,1.416667,207.410417,96.750000,2.0
std,0.0,0.467177,18.073017,0.954521,1.130739,58.509153,36.782016,0.0
min,1.0,0.000000,11.000000,0.000000,0.000000,78.850000,45.000000,2.0
25%,1.0,0.000000,22.500000,0.000000,0.000000,164.866700,83.250000,2.0
50%,1.0,1.000000,33.750000,1.000000,2.000000,221.779200,109.000000,2.0
75%,1.0,1.000000,50.000000,1.000000,2.000000,262.375000,122.500000,2.0
max,1.0,1.000000,76.000000,3.000000,4.000000,263.000000,124.000000,2.0


You may have noticed group 1 has a 100% survival rate, and group 3's survival rate was as low as 10%. As we can see, the model was able to group the passengers in a way that can help us determine what factors led to death or survival, all without us programming it to do so.